In [42]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
%matplotlib inline
np.random.seed(1) # set a seed so that the results are consistent

In [55]:
x1 = np.random.randn(50000,1)
x1 = x1[x1>0].astype(int)
x2 = np.random.randn(50000,1)
x2 = x2[x2>0].astype(int)

n = min(x1.shape[0], x2.shape[0])
x1 = x1[0:n]
x2 = x2[0:n]
x3 = np.bitwise_xor(x1,x2)
X = np.stack((x1, x2), axis = 0)
Y = x3.reshape((1,x3.shape[0]))

[0 1 2 3 5 6]


AttributeError: 'numpy.ndarray' object has no attribute 'unique'